In [ ]:
    pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 92.6 MB/s eta 0:00:00


In [ ]:
pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.2 MB/s eta 0:00:00


In [ ]:
import onnx, onnxruntime as ort
print("onnx:", onnx.__version__, "onnxruntime:", ort.__version__)


onnx: 1.19.0 onnxruntime: 1.22.1


In [ ]:
"""
Linear Regression -> ONNX (no scikit-learn)

- Reads parameters from params.json: { "intercept": float, "coefficient": [..], "covariance": [[..], ..] }
- Builds an ONNX model: y = X @ W + b
- Stores covariance in ONNX metadata
- Generates synthetic test data and validates with onnxruntime

Required: pip install onnx onnxruntime numpy
(Optional): pip install pandas if you later want to load real CSV/XLS data
"""

import os, json
import numpy as np
import onnx
from onnx import helper, TensorProto, numpy_helper
import onnxruntime as ort


PARAMS_PATH = "params.json"
ONNX_PATH = "linear_regression.onnx"
OPSET = 11          # was 13; MatMul/Add are fine in 11
IR_VERSION = 10      # <= your runtime’s max (10). 7 is very safe.



def ensure_dummy_params(path: str):
    """Create a dummy params.json if not present."""
    if os.path.exists(path):
        return
    params = {
        "intercept": 1.5,
        "coefficient": [2.0, -3.0, 0.5],  # D = 3 features
        "covariance": [
            [0.04, 0.00, 0.00],
            [0.00, 0.09, 0.00],
            [0.00, 0.00, 0.01],
        ]
    }
    with open(path, "w") as f:
        json.dump(params, f, indent=2)


def load_params(path: str):
    with open(path, "r") as f:
        p = json.load(f)
    intercept = float(p["intercept"])
    coef = np.asarray(p["coefficient"], dtype=np.float32)  # shape (D,)
    cov = np.asarray(p["covariance"], dtype=np.float32)    # shape (D, D)
    assert cov.shape == (coef.shape[0], coef.shape[0]), "Covariance must be DxD for D coefficients."
    return intercept, coef, cov


def build_onnx_linear(onnx_path: str, intercept: float, coef: np.ndarray, covariance: np.ndarray):
    """
    Create an ONNX graph implementing y = X @ W + b
    X: [N, D] float
    W: [D, 1] float
    b: [1]     float
    y: [N, 1]  float
    """
    D = coef.shape[0]

    # IO value infos
    X = helper.make_tensor_value_info('X', TensorProto.FLOAT, ['N', D])
    Y = helper.make_tensor_value_info('y', TensorProto.FLOAT, ['N', 1])

    # Initializers (weights and bias)
    W_np = coef.reshape(D, 1).astype(np.float32)         # [D,1]
    b_np = np.array([intercept], dtype=np.float32)       # [1]

    W_init = numpy_helper.from_array(W_np, name="W")
    b_init = numpy_helper.from_array(b_np, name="b")

    # Nodes: MatMul(X, W) -> Z ; Add(Z, b) -> y
    matmul = helper.make_node("MatMul", ["X", "W"], ["Z"], name="MatMul_XW")
    add    = helper.make_node("Add",    ["Z", "b"], ["y"], name="Add_Bias")

    graph = helper.make_graph(
        nodes=[matmul, add],
        name="LinearRegression",
        inputs=[X],
        outputs=[Y],
        initializer=[W_init, b_init]
    )

    model = helper.make_model(
    graph,
    opset_imports=[helper.make_operatorsetid("", OPSET)],
    producer_name="custom-linear-no-sklearn",
    ir_version=IR_VERSION,     # <-- important
   )


    # Attach covariance matrix as JSON in metadata (not used for computation)
    meta = model.metadata_props.add()
    meta.key = "covariance_json"
    meta.value = json.dumps(covariance.tolist())

    # (Optional) also drop covariance as a non-consumed initializer tensor for provenance
    cov_init = numpy_helper.from_array(covariance.astype(np.float32), name="covariance_matrix")
    model.graph.initializer.append(cov_init)

    # Validate & save
    onnx.checker.check_model(model)
    onnx.save(model, onnx_path)
    return onnx_path


def generate_synthetic_data(coef: np.ndarray, intercept: float, n: int = 200, noise_std: float = 0.3, seed: int = 42):
    rng = np.random.default_rng(seed)
    D = coef.shape[0]
    X = rng.normal(size=(n, D)).astype(np.float32)
    y = (X @ coef.astype(np.float32)) + float(intercept) + rng.normal(scale=noise_std, size=n).astype(np.float32)
    return X, y


def run_onnx_inference(onnx_path: str, X: np.ndarray) -> np.ndarray:
    sess = ort.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])
    input_name = sess.get_inputs()[0].name
    (y_pred,) = sess.run(None, {input_name: X})  # shape [N,1]
    return y_pred.reshape(-1)


def rmse(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))


def mae(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return float(np.mean(np.abs(y_true - y_pred)))


def r2_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    ss_res = float(np.sum((y_true - y_pred) ** 2))
    ss_tot = float(np.sum((y_true - np.mean(y_true)) ** 2))
    return 1.0 - ss_res / ss_tot if ss_tot > 0 else 0.0


In [ ]:
PARAMS_PATH = "params.json"
ONNX_PATH = "linear_regression.onnx"

ensure_dummy_params(PARAMS_PATH)
intercept, coef, covariance = load_params(PARAMS_PATH)
print("intercept:", intercept)
print("coef shape:", coef.shape, "coef:", coef)
print("cov shape:", covariance.shape)


intercept: 1.5
coef shape: (3,) coef: [ 2.  -3.   0.5]
cov shape: (3, 3)


In [ ]:
onnx_file = build_onnx_linear(ONNX_PATH, intercept, coef, covariance)
print("Saved:", onnx_file)

m = onnx.load(onnx_file)
print({kv.key: kv.value[:120] + "..." for kv in m.metadata_props})  # peek metadata


Saved: linear_regression.onnx
{'covariance_json': '[[0.03999999910593033, 0.0, 0.0], [0.0, 0.09000000357627869, 0.0], [0.0, 0.0, 0.009999999776482582]]...'}


In [ ]:
X_test, y_true = generate_synthetic_data(coef, intercept, n=500, noise_std=0.25, seed=123)
X_test.shape, y_true.shape


((500, 3), (500,))

In [ ]:
y_pred = run_onnx_inference(onnx_file, X_test)
print("y_pred shape:", y_pred.shape)
print("first 5 preds:", y_pred[:5])


y_pred shape: (500,)
first 5 preds: [ 1.2690798  -0.58419204 -1.5570817  -0.19924545  6.3977356 ]


In [ ]:
print("RMSE:", rmse(y_true, y_pred))
print("MAE :", mae(y_true, y_pred))
print("R^2 :", r2_score(y_true, y_pred))


RMSE: 0.2563903033733368
MAE : 0.20465616881847382
R^2 : 0.9947278904788079


In [ ]:
y_np = (X_test @ coef.astype(np.float32)) + float(intercept)
print("Max |NumPy - ONNX|:", np.max(np.abs(y_np - y_pred)))


Max |NumPy - ONNX|: 9.536743e-07


In [ ]:
# Example: change intercept and one coefficient, then rebuild & re-eval
intercept += 0.2
coef[0] += 0.1
build_onnx_linear(ONNX_PATH, intercept, coef, covariance)
y_pred = run_onnx_inference(ONNX_PATH, X_test)
print("R^2 (after tweak):", r2_score(y_true, y_pred))


R^2 (after tweak): 0.9907701856596979
